# Importing libraries

In [3]:
# 'https://hilpisch.com/tr_eikon_eod_data.csv'
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'
plt.style.use('seaborn-v0_8')
# print(plt.style.available)

In [4]:
import numpy as np
import pandas as pd
import scipy.optimize as sco

# 1 - Vasicek stochastic interest rate model

Stochastic interest rate models can capture the random behaviour of interest rates. The Vasicek interest rate model is the first stochastic interest rate model introduced in quantitative finance.

The equation governing the inrest rate in the CIR model is given by
\begin{align}
d r_t & = a (b-r_t) dt + \sigma dW_t\,,
\end{align}
where $r_t$ is interest rate at time $t$, $a$ is the speed of mean reversion (which specifies how fast the interest rate returns to the mean), $b$ is the long term average interest rate, $dt$ is usually $1/252$ (252 is the average number of trading days in US), $\sigma$ is the volatility of interest rates and $dW_t$ is the Brownian motion or Wiener process.

The randomness or unpredictability in the interest rates is modeled by the Brownian motion which is given by $dW_t = x_t \sqrt{dt}$, where $x_t$ is a random number generated from a normal distribution.

Note that the interest rates can fluctuate by economic news, market sentiment, policy change, and this behaviour is captured by the Brownian motion contribution $dW_t$, which is also called the stochastic part.

Note that this model admits negative interest rates, so it can be a good fit for countries like Germany, Japan, etc. It is also a single factor model, which means that market risk $dW_t$ is the only part affecting the interest rate.

The parameters of the model are adjusted in such a way that the interest rate predicted by the Vasicek model closely aligns with the available data.

# 2 - Cox-Ingersoll-Ross stochastic interest rate model

Similar to the NS/NSS models, the Cox-Ingersoll-Ross (CIR) model is used to model the interest rate moments. The CIR model is also called one factor model, since it describes interest rate movement as driven by a sole function of market risk.

The equation governing the inrest rate in the CIR model is given by
\begin{align}
d r_t & = a (b-r_t) dt + \sigma \sqrt{r_t} dW_t\,,
\end{align}
where $r_t$ is the interst rate, $a$ is the speed of mean reversion (i.e., the speed that the interest rate reaches the mean), $b$ is the mean interest rate, $\sigma$ is the volatility and $dW_t$ is the Brownian motion (or Wiener process).

Let us analyze a few cases. Note that if $b > r_t $ there will be a positive movement, if $b < r_t$ there will be a negative movement. In addition, note that the Brownian motion term has vanishing mean.

The CIR model assumes mean reversion: interests rates tend to move towards long term equilibrium $b$. It also assumes that the parameters $\{a,b,\sigma\}$ are constant. However, note that these parameters might change in time due to economic and market conditions. It also assumes a continuous path for interest rates.

The Vasicek model does not have the term $\sqrt{r_t}$ multiplying the Brownian motion contribution $dW_t$ when compared to the CIR model, which means that the interest rates are always positive in the CIR model. This means that when modeling the US yield curve the CIR model will be preferred. On the other hand, when modeling the yield curve in Germany the Vasicek model will be the best choice.

Therefore, the CIR model will be the best fit when interest rates are always positive and the Vasicek model will be the best fit when interest rates are both positive and negative.


# 2 - Implementation

We first download the US treasury data for different maturities.

In [14]:
df_original = pd.read_csv('daily-treasury-rates_2023.csv')
df0 = df_original.copy()

In [15]:
df0.head()

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,12/29/2023,5.60,5.59,5.40,5.41,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.20,4.03
1,12/28/2023,5.57,5.55,5.45,5.42,5.28,4.82,4.26,4.02,3.83,3.84,3.84,4.14,3.98
2,12/27/2023,5.55,5.53,5.44,5.42,5.26,4.79,4.20,3.97,3.78,3.81,3.79,4.10,3.95
3,12/26/2023,5.53,5.52,5.45,5.44,5.28,4.83,4.26,4.05,3.89,3.91,3.89,4.20,4.04
4,12/22/2023,5.54,5.52,5.44,5.45,5.31,4.82,4.31,4.04,3.87,3.92,3.90,4.21,4.05


In [16]:
df0.tail()

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
245,01/09/2023,4.37,4.58,4.70,4.74,4.83,4.69,4.19,3.93,3.66,3.60,3.53,3.83,3.66
246,01/06/2023,4.32,4.55,4.67,4.74,4.79,4.71,4.24,3.96,3.69,3.63,3.55,3.84,3.67
247,01/05/2023,4.30,4.55,4.66,4.75,4.81,4.78,4.45,4.18,3.90,3.82,3.71,3.96,3.78
248,01/04/2023,4.20,4.42,4.55,4.69,4.77,4.71,4.36,4.11,3.85,3.79,3.69,3.97,3.81
249,01/03/2023,4.17,4.42,4.53,4.70,4.77,4.72,4.40,4.18,3.94,3.89,3.79,4.06,3.88
